In [20]:
import autograd.numpy as anp
from numpy import random as rnd
from numpy import linalg as la 

import pymanopt
from pymanopt.manifolds import Grassmann
from pymanopt.solvers import TrustRegions

This example generates a random 128 x 128 symmetric matrix, and finds the dominant invariant 3-dimensional subspace for this matrix.
That is, it finds the subspace spanned by the three eigenvectors with the largest eigenvalues

In [21]:
def create_cost_egrad_ehess(manifold, matrix):
	egrad = ehess = None
	@pymanopt.function.Autograd
	def cost(X):
		return -anp.trace(X.T @ matrix @ X)
	return cost, egrad, ehess

In [22]:
num_rows = 4
subspace_dimension = 2
matrix = rnd.randn(num_rows, num_rows)
matrix = 0.5* (matrix + matrix.T)

In [23]:
manifold = Grassmann(num_rows, subspace_dimension)
cost, egrad, ehess = create_cost_egrad_ehess(manifold, matrix)
problem = pymanopt.Problem(manifold, cost=cost, egrad=egrad, ehess=ehess)

In [24]:
solver = TrustRegions()
estimated_spanning_set = solver.solve(problem, Delta_bar=8*anp.sqrt(subspace_dimension))

Optimizing...
                                            f: +1.210103e+00   |grad|: 2.640284e+00
acc       k:     1     num_inner:     0     f: -7.451128e-01   |grad|: 3.845104e-01   negative curvature
REJ TR-   k:     2     num_inner:     1     f: -7.451128e-01   |grad|: 3.845104e-01   negative curvature
acc TR+   k:     3     num_inner:     0     f: -8.367867e-01   |grad|: 1.292002e+00   exceeded trust region
acc       k:     4     num_inner:     1     f: -1.312656e+00   |grad|: 1.169599e+00   negative curvature
acc       k:     5     num_inner:     2     f: -1.528690e+00   |grad|: 6.141880e-01   exceeded trust region
acc       k:     6     num_inner:     2     f: -1.560247e+00   |grad|: 1.843784e-02   reached target residual-kappa (linear)
acc       k:     7     num_inner:     3     f: -1.560296e+00   |grad|: 5.201502e-07   reached target residual-theta (superlinear)
Terminated - min grad norm reached after 7 iterations, 0.02 seconds.



In [25]:
eigenvalues, eigenvectors = la.eig(matrix)
column_indices = anp.argsort(eigenvalues)[-subspace_dimension:]
spanning_set = eigenvectors[:, column_indices]
print(
	"Geodesic distance between true and estimated dominant subspace:",
	manifold.dist(spanning_set, estimated_spanning_set),
)


Geodesic distance between true and estimated dominant subspace: 1.548574096771072e-07


In [26]:
spanning_set

array([[ 0.60353026, -0.49524432],
       [ 0.13041879,  0.78745011],
       [-0.57261132, -0.36610697],
       [ 0.53931293, -0.02492124]])

In [27]:
estimated_spanning_set

array([[-0.47865382, -0.61677087],
       [ 0.79070462, -0.10896305],
       [-0.38153854,  0.56244676],
       [-0.01025021, -0.53979107]])